In [3]:
import os
import pandas as pd
import numpy as np
import pickle
from matplotlib.image import imread
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

data_dev = []
data_test = []
data_train = []
file_name_dev = "audioDataDev.pickle"
file_name_test = "audioDataTest.pickle"
file_name_train = "audioDataTrain2000.pickle"

with (open(file_name_dev, "rb")) as f:
    while True:
        try:
            data_dev.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_test, "rb")) as f:
    while True:
        try:
            data_test.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_train, "rb")) as f:
    while True:
        try:
            data_train.append(pickle.load(f))
        except EOFError:
            break
f.close()

In [ ]:
len(data_dev) +2

1874

In [5]:
X_dev = []
y_dev = []
X_train = []
y_train = []
X_test = []
y_test = []

for i in range(len(data_dev)-1):
    X_dev.append(data_dev[i+1][3])
for i in range(len(data_dev)-1):
    y_dev.append(data_dev[i+1][5:12])
    
for i in range(len(data_test)-1):
    X_test.append(data_test[i+1][3])
for i in range(len(data_test)-1):
    y_test.append(data_test[i+1][5:12])
    
for i in range(len(data_train)-1):
    X_train.append(data_train[i+1][3])
for i in range(len(data_train)-1):
    y_train.append(data_train[i+1][5:12])

X_dev = np.array(X_dev)
y_dev = np.array(y_dev)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_train = np.array(X_train)
y_train = np.array(y_train)


In [6]:
X_train.shape

(598, 128, 850)

In [7]:
X_dev = X_dev.reshape(X_dev.shape[0], 128, 850, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 850, 1)
X_train = X_train.reshape(X_train.shape[0], 128, 850, 1)

In [8]:
X_train.shape

(598, 128, 850, 1)

In [9]:
# get the model
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(128, 850, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='sigmoid'))
    return model
 
# evaluate a model using repeated k-fold cross-validation
    
model = get_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)
yhat = model.predict(X_test)

In [ ]:
print(yhat)
yhat = yhat.round()
print(yhat)
print(y_test)

[[0.43790525 0.43161246 0.4189765  0.53402627 0.4353226  0.4244436
  0.45513678]
 [0.4371715  0.4308189  0.4182269  0.5343549  0.4347349  0.4237163
  0.45472223]
 [0.44143257 0.43586525 0.42417347 0.5320065  0.43970272 0.4293132
  0.45829058]
 [0.4353499  0.42869022 0.41520587 0.53561693 0.4325964  0.42118472
  0.45321527]
 [0.43810835 0.43189386 0.41939864 0.5339438  0.4357423  0.4249166
  0.45531377]
 [0.44022104 0.4342802  0.42256674 0.5327049  0.4379388  0.4276873
  0.45713603]]
[[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]
[[0 0 2 3 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 2]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


In [ ]:
total = 0
similar = 0
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        if y_test[i][j] == yhat[i][j]:
            similar += 1
        total += 1
acc = similar/total

print('>%.3f' % acc)
results.append(acc)

>0.810
